#### This notebook is apply Discrete-FDR to identify significant features (reference: Jiang et al, msystems, 2017)

In [1]:
import numpy as np
import pandas as pd
from biom import load_table
from gneiss.util import match

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from dsfdr import dsfdr
from dsfdr import simulation
from dsfdr import statistics
from dsfdr import transform

In [3]:
# set seed to obtain consistent result
np.random.seed(2018)

## Prepare biom table and mapping file

### load biom table

In [11]:
def convert_biom_to_pandas(table):
    otu_table = pd.DataFrame(np.array(table.matrix_data.todense()).T,
                             index=table.ids(axis='sample'),
                             columns=table.ids(axis='observation'))
    return otu_table

In [12]:
table = load_table('./data/haddad_6weeks_allFeatures_pqn_matched.biom')
table = convert_biom_to_pandas(table)

In [13]:
table = table.T

In [14]:
table.shape

(182, 1710)

### load mapping file

In [17]:
mapping = pd.read_table("./data/haddad_6weeks_metadata_matched.txt", sep='\t', header=0, index_col=0)

In [18]:
mapping.shape

(182, 69)

In [19]:
mapping.head()

,BarcodeSequence,LinkerPrimerSequence,center_name,experiment_design_description,extraction_robot,extractionkit_lot,instrument_model,library_construction_protocol,linker,mastermix_lot,...,physical_specimen_location,physical_specimen_remaining,sample_type,scientific_name,sex,title,weekly_cage_food_consumption,weight,weight_units,Description
#SampleID,,,,,,,,,,,,,,,,,,,,,
10422.17.F.10,GTTGTTCTGGGA,GTGTGCCAGCMGCCGCGGTAA,UCSDMI,Mouse cohort exposed to apnea and controls to ...,HOWE_KF1,PM16B24,Illumina HiSeq 2500,"EMP 16S V4 protocol 515fbc, 806r",GT,14663,...,UCSD LBR -80 freezer,True,stool,mouse gut metagenome,male,OSA,Missing: Not provided,25.6,g,feces mouse 17 collection 10 of 13
10422.17.F.11,TGTGCTTGTAGG,GTGTGCCAGCMGCCGCGGTAA,UCSDMI,Mouse cohort exposed to apnea and controls to ...,HOWE_KF2,PM16B24,Illumina HiSeq 2500,"EMP 16S V4 protocol 515fbc, 806r",GT,14663,...,UCSD LBR -80 freezer,True,stool,mouse gut metagenome,male,OSA,71.8,25.2,g,feces mouse 17 collection 11 of 13
10422.17.F.12,AGAATCCACCAC,GTGTGCCAGCMGCCGCGGTAA,UCSDMI,Mouse cohort exposed to apnea and controls to ...,HOWE_KF1,PM16B24,Illumina HiSeq 2500,"EMP 16S V4 protocol 515fbc, 806r",GT,14663,...,UCSD LBR -80 freezer,True,stool,mouse gut metagenome,male,OSA,Missing: Not provided,25.7,g,feces mouse 17 collection 12 of 13
10422.17.F.13,CTGTAAAGGTTG,GTGTGCCAGCMGCCGCGGTAA,UCSDMI,Mouse cohort exposed to apnea and controls to ...,HOWE_KF2,PM16B24,Illumina HiSeq 2500,"EMP 16S V4 protocol 515fbc, 806r",GT,14663,...,UCSD LBR -80 freezer,True,stool,mouse gut metagenome,male,OSA,71.7,26.3,g,final feces mouse 17 collection 13 of 13
10422.17.F.3,CTCCCGAGCTCC,GTGTGCCAGCMGCCGCGGTAA,UCSDMI,Mouse cohort exposed to apnea and controls to ...,HOWE_KF2,PM16B24,Illumina HiSeq 2500,"EMP 16S V4 protocol 515fbc, 806r",GT,14663,...,UCSD LBR -80 freezer,True,stool,mouse gut metagenome,male,OSA,105.7,24.9,g,feces mouse 17 collection 3 of 13


In [20]:
mapping.exposure_type.value_counts()

IHH    92
Air    90
Name: exposure_type, dtype: int64

### match mapping file and biom table

In [22]:
mapping, table = match(mapping, table)

In [23]:
print(mapping.shape)
print(table.shape)

(182, 69)
(182, 1710)


In [24]:
# convert values in exposure_type to be integers
labels = np.array((mapping['exposure_type'] == 'IHH').astype(int))

In [25]:
labels

array([1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0])

In [28]:
# transpose otu_table to have OTUs as rows and samples as columns
dat = np.transpose(np.array(table))

In [29]:
dat.shape

(1710, 182)

## Apply DS-FDR

In [30]:
result = dsfdr.dsfdr(dat, labels, transform_type='rankdata', method='meandiff', alpha=0.01, 
                     numperm=10000, fdr_method='dsfdr')

In [31]:
rej = result[0]
# total number of significant hypotheses
np.sum(rej)

382

## Output results

In [32]:
pvals=[]
teststat=[]

for i in range(len(result[0])):
    if result[0][i]==True:
        pvals.append(result[2][i])
        teststat.append(result[1][i])

In [33]:
s = pd.Series(rej, name='bools')

In [35]:
out = table.T[s.values]

In [36]:
out.head()

,10422.24.F.5,10422.29.F.6,10422.27.F.8,10422.26.F.12,10422.27.F.13,10422.23.F.13,10422.26.F.4,10422.28.F.9,10422.26.F.3,10422.17.F.4,...,10422.26.F.10,10422.25.F.8,10422.17.F.9,10422.18.F.4,10422.26.F.8,10422.18.F.12,10422.24.F.12,10422.25.F.11,10422.24.F.6,10422.29.F.5
357.2784138555112_5.010000161030595,6.917097e+08,2.578800e+08,1.910583e+08,4.225667e+08,3.769354e+08,4.891753e+08,2.875788e+08,1.809306e+08,2.760070e+08,6.040222e+08,...,2.986456e+08,1.467964e+08,4.736494e+08,6.646255e+08,2.369384e+08,4.067838e+08,5.849852e+08,1.258402e+08,4.206568e+08,5.217787e+08
104.1070008277893_0.3359217147435899,3.281495e+08,5.548085e+08,2.555961e+08,4.317716e+08,5.935661e+08,4.738261e+08,2.641577e+08,8.133259e+08,3.024110e+08,2.319210e+08,...,6.051821e+08,3.286935e+08,6.226845e+08,2.803015e+08,4.983045e+08,3.681432e+08,3.707433e+08,4.512311e+08,3.555671e+08,3.888311e+08
373.27348764725264_4.338406810897439,5.235827e+08,4.383068e+08,1.401338e+08,2.872223e+08,1.766426e+08,5.204261e+08,5.735200e+08,1.213430e+08,6.130434e+08,3.872113e+08,...,2.466242e+08,7.303587e+07,5.619251e+08,8.967402e+07,1.272761e+08,4.332568e+08,6.157758e+08,5.819768e+07,5.773077e+08,6.269518e+08
391.2841105382007_4.130280368589744,5.955825e+08,3.276695e+08,1.711783e+08,3.190921e+08,1.631000e+08,6.351529e+08,4.515842e+08,2.888728e+08,4.180027e+08,4.944621e+08,...,2.666427e+08,1.333114e+08,5.289577e+08,6.775269e+08,1.772450e+08,4.389572e+08,5.313699e+08,9.497437e+07,3.838284e+08,5.565597e+08
283.2623651476874_5.462460305958134,8.325900e+08,1.365513e+08,2.375455e+08,6.955653e+08,6.338738e+08,6.047327e+08,2.618255e+08,1.065081e+09,3.702029e+08,3.005478e+08,...,5.033927e+08,3.872046e+08,1.671962e+08,2.699610e+08,5.791108e+08,7.148008e+07,4.633385e+08,7.710580e+08,3.691103e+08,4.800439e+08


In [37]:
out.shape

(382, 182)

In [38]:
out['pvalue']=pvals
out['test_statistic']=teststat

In [39]:
out.to_csv('./data/allFeatures_dsfdr.txt', sep='\t')